In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

## 4: Multiplying Matrices
- test that $\sigma_j\sigma_k = \delta_{jk}\mathbf{1} + i \epsilon_{jkl}\sigma_l$

In [2]:
#from 2024-1
def norm_diff(L, M):
    LminusM = L - M
    mag_LminusM = np.sqrt(np.sum(np.abs(LminusM)**2))
    mag_L = np.sqrt(np.sum(np.abs(L)**2))
    mag_M = np.sqrt(np.sum(np.abs(M)**2))
    
    delta = mag_LminusM / (mag_L + mag_M)
    
    return delta

## 5: Create function to multiply matrices
- (5a): MAKE GENERAL MATRIX MAKER: create function that inputs 2 arrays of complex numbers ($A_0, \vec{A}$) and ($B_0, \vec{B}$) and output array of complex numbers: ($C_0, \vec{C}$)
- (5b) create function that outputs array of 4 random complex numbers
- (5c) Test function using random A and B arrays compare two ways

In [3]:
##5a: make general matrix maker
def matrix_maker(m0xyz): #array to matrix
    M0, Mx, My, Mz = m0xyz
    
    identity_matrix = np.zeros((2,2), dtype = 'complex128')
    identity_matrix[0,0]= 1
    identity_matrix[1,1] = 1

    pauli_x = np.zeros((2,2), dtype = 'complex128')
    pauli_x[0,1]= 1
    pauli_x[1,0] = 1

    pauli_y = np.zeros((2,2), dtype = 'complex128')
    pauli_y[0,1]= -1j
    pauli_y[1,0] = 1j

    pauli_z = np.zeros((2,2), dtype = 'complex128')
    pauli_z[0,0]= 1
    pauli_z[1,1] = -1
    
    m = np.array([Mx, My, Mz])
    
    pauli_basis = np.array([pauli_x, pauli_y, pauli_z])
    
    m_dot_pauli = np.tensordot(m, pauli_basis, axes=1)
    
    matrix = M0 * identity_matrix + m_dot_pauli
    
    return matrix

In [4]:
a0xyz = [1, 1, 1, 1]
b0xyz = [2, 2, 2, 2]
A = matrix_maker(a0xyz)
B = matrix_maker(b0xyz)
C = A@B
print(A)
print(B)
print(C)

[[2.+0.j 1.-1.j]
 [1.+1.j 0.+0.j]]
[[4.+0.j 2.-2.j]
 [2.+2.j 0.+0.j]]
[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]


In [5]:
## takes two arrays and creates a third that can then be used to make a matrix
def calculate_c0xyz(a0xyz, b0xyz): #arrays to array
    A0, Ax, Ay, Az = a0xyz
    B0, Bx, By, Bz = b0xyz
    axyz = np.array([Ax, Ay, Az])
    bxyz = np.array([Bx, By, Bz])
    C0 = A0*B0 + np.tensordot(axyz, bxyz, axes=1)
    cxyz = A0*bxyz + B0*axyz +1j*(np.cross(axyz, bxyz))
    c0xyz = ([C0, cxyz[0], cxyz[1], cxyz[2]])
    return c0xyz

In [6]:
#testing calculate_c0xyz
e0xyz = [1, 1, 1, 1]
f0xyz = [2, 2, 2, 2]
g0xyz = calculate_c0xyz(e0xyz, f0xyz)
print(g0xyz)
G = matrix_maker(g0xyz)
print(G)

[8, (4+0j), (4+0j), (4+0j)]
[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]


In [7]:
##5b make random number generator
    ## np.random.randn(n) returns a 1D array with n random numbers, so here it makes a 1D array of 4 random numbers (could make a matrix of random numbers or a 2D array i should say, if you give two inputs like np.random.randn(m,n))
    ## and "randn" refers to "random normal" which means it generates numbers from the standard normal distribution (gaussian distribution)
def random_complex_numbers():
    real_parts = np.random.randn(4)
    imaginary_parts = np.random.randn(4)
    complex_numbers = real_parts + 1j*imaginary_parts
    return complex_numbers

random_numbers = random_complex_numbers()
print(random_numbers)

[-1.90035134-1.08695114j  0.5390131 -0.03510656j -1.99795283-0.98494175j
 -0.80295687+0.79802329j]


In [8]:
#take a0xyz and b0xyz and make c0xyz then C
def one_matrix_test(a0xyz, b0xyz): #arrays to matrix
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    C = matrix_maker(c0xyz)
    return C

#take a0xyz and b0xyz and make A and B then C
def three_matrix_test(a0xyz, b0xyz): #arrays to matrix
    A = matrix_maker(a0xyz)
    B = matrix_maker(b0xyz)
    C = A@B
    return C

In [9]:
#testing
h0xyz = [1, 1, 1, 1]
i0xyz = [2, 2, 2, 2]
C1 = one_matrix_test(h0xyz, i0xyz)
C2 = three_matrix_test(h0xyz, i0xyz)

In [10]:
norm_diff(C1, C2)
print(C1)
print (C2)

[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]
[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]


In [11]:
### 5c: testing function using random numbers
def trial_loop(N): 

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()

        #compute density matrix both ways
        L = one_matrix_test(random_numbers1, random_numbers2)
        M = three_matrix_test(random_numbers1, random_numbers2)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2)) #called it something different because i was trying to append random numbers to itslef and it said no
    
    return delta_values, random_numbers_used

In [12]:
N = 100
delta_values, random_numbers_used = trial_loop(N) #run like this to capture outputs

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [6.167707824773343e-17, 5.58998071976527e-17, 9.215547674805432e-17, 5.2059115316788976e-17, 1.100156462596451e-16, 6.221396304481016e-17, 9.295866041650736e-17, 1.1655280150238518e-16, 5.4899456550719054e-17, 4.5374124881462027e-17, 1.41062944459761e-16, 7.70215878270881e-17, 6.109226461232317e-17, 1.236874987054378e-16, 8.819119599767934e-17, 4.7674340701805317e-17, 5.021250605572257e-17, 5.0827291953829304e-17, 3.4063339095088086e-17, 1.0238629897069152e-16, 4.825571609845867e-17, 9.64888984224041e-17, 6.05026804992959e-17, 9.806667611968916e-17, 1.1990688635880384e-16, 6.427491197201496e-17, 6.578467699307292e-17, 8.03700788742841e-17, 5.665560667351814e-17, 7.293064848754989e-17, 8.568921436146492e-17, 3.7322039578176634e-17, 8.065582430428329e-17, 8.503497920157619e-17, 1.06117257601425e-16, 5.4956113691973864e-17, 9.358318948265054e-17, 6.305945106408007e-17, 4.371748326286116e-17, 3.059549407577645e-17, 8.766106784372411e-17, 7.629892530252834e-17, 7.6687872406774

## 6: Create matrix multiplication for $(\mathbf{1} - \rho^{(2)})\rho^{(4)}$
- (6a): create functions that input $(P_0, \vec{P})$ and output $(A_0, \vec{A})$ and $(B_0, \vec{B})$
- (6b): test

In [13]:
## 6a
def array_for_rho4(p0xyz): 
    P0, Px, Py, Pz = p0xyz
    pxyz = np.array([Px, Py, Pz])
    B0 = 0.5*P0
    bxyz = 0.5*P0*pxyz
    b0xyz = ([B0, bxyz[0], bxyz[1], bxyz[2]])
    return b0xyz #makes array to be put into matrix maker to create rho

def array_for_rho2(p0xyz):
    P0, Px, Py, Pz = p0xyz
    pxyz = np.array([Px, Py, Pz])
    A0 = 1 - 0.5*P0
    axyz = -0.5*P0*pxyz
    a0xyz = ([A0, axyz[0], axyz[1], axyz[2]])
    return a0xyz
#i think the point of this is to take the specific array and matrix and make it general so I can put into general matrix maker. If i was using create_density_matrix, i could probably put p0xyz directly in and it would make the matrix?

^ i think the point of this is to take the specific array and matrix and make it general so I can put into general matrix maker. If i was using create_density_matrix, i could probably put p0xyz directly in and it would make the matrix?

In [14]:
## 6b testing
p0xyz = [1, 1, 1, 1]
b0xyz = array_for_rho4(p0xyz) #how to check this
print('ARRAY B:', b0xyz)
rho4 = matrix_maker(b0xyz)
print('RHO4:',rho4)

p0xyz = [1, 1, 1, 1]
a0xyz = array_for_rho2(p0xyz) #how to check this
print('ARRAY A:', a0xyz)
rho2 = matrix_maker(a0xyz)
print('RHO2:', rho2)

ARRAY B: [0.5, 0.5, 0.5, 0.5]
RHO4: [[1. +0.j  0.5-0.5j]
 [0.5+0.5j 0. +0.j ]]
ARRAY A: [0.5, -0.5, -0.5, -0.5]
RHO2: [[ 0. +0.j  -0.5+0.5j]
 [-0.5-0.5j  1. +0.j ]]


## 7: Calculate $ C = (\mathbf{1} - \rho^{(2)})\rho^{(4)} = C_0\mathbf{1} + \vec{C}* \vec{\sigma}$
- (7a): create function that inputs 2 arrays ($P_0^{(2)}, \vec{P^{(2)}}$) and ($P_0^{(4)}, \vec{P^{(4)}}$) and outputs $(C_0, \vec{C})$
- (7b): test

- $\textbf{create_rho2}$ creates an array that can be put into matrix maker to create matrix "rho2"
- $\textbf{create_rho4}$ creates an array that can be put into matrix maker to create matrix "rho4"

- "rho2" is a matrix:  $(\mathbf{1} - \rho^{(2)})$
- "rho4" is a matrix: $\rho^{(4)}$

- so to create rho2 or rho4, need to take p2 or p4 and input it into the correct function, then we will have an array to put into matrix maker to get rho2 and rho4?
..... i am getting lost

In [15]:
#### dont need this: do need this!!!
def rho2_matrix(p0xyz_2):
    rho2_array = array_for_rho2(p0xyz_2)
    rho2 = matrix_maker(rho2_array)
    return rho2

def rho4_matrix(p0xyz_4):
    rho4_array = array_for_rho4(p0xyz_4)
    rho4 = matrix_maker(rho4_array)
    return rho4

In [16]:
##7a
def rho2timesrho4(p0xyz_2, p0xyz_4):
    #make a0xyz and b0xyz from p0xyz_2 and p0xyz_4 -- thats what array_for_rho4 functions do!
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    return c0xyz

p0xyz_2 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]
c10xyz = rho2timesrho4(p0xyz_2, p0xyz_4)
C1 = matrix_maker(c10xyz)
print(c10xyz)
print(C1)

[-0.5, 0j, 0j, 0j]
[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]


COMPARING FUNCTIONS:
- function 1, $\textbf{final_C_1}$ is taking in p2 and p4 arrays, creating a0xyz and b0xyz arrays to then get a third array c0xyz which can go into matrix maker and get the final matrix C
- function 2, $\textbf{final_C_2}$ is taking in p2 and p4 arrays and making rho2 and rho4 matrices to then be multiplied and create matrix C


In [17]:
##7b: comparing functions:
%time
def final_C_1(p0xyz_2, p0xyz_4):
    #make a0xyz and b0xyz from p0xyz_2 and p0xyz_4 -- thats what array_for_rho4 functions do!
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    C1 = matrix_maker(c0xyz)
    return C1
%time
def final_C_2(p0xyz_2, p0xyz_4):
    rho2 = rho2_matrix(p0xyz_2)
    rho4 = rho4_matrix(p0xyz_4)
    C2 = rho2@rho4
    return C2

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 2.62 µs
CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 2.86 µs


In [18]:
p0xyz_2 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]
C1 = final_C_1(p0xyz_2, p0xyz_4)
C2 = final_C_2(p0xyz_2, p0xyz_4)
print (C1)
print (C2)

[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]
[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]


In [19]:
### testing with random numbers
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        
        L = final_C_1(random_numbers1, random_numbers2)
        M = final_C_2(random_numbers1, random_numbers2)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2)) #called it something different because i was trying to append random numbers to itslef and it said no
    
    return delta_values, random_numbers_used

In [20]:
N = 100 
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [6.860534557743019e-17, 1.2295180221334533e-16, 9.540314019715582e-17, 1.0570935190503355e-16, 8.815917513787434e-17, 8.880294643786024e-17, 8.987278823448166e-17, 7.509201224487741e-17, 1.3059258151845015e-16, 7.16162054965643e-17, 9.890627886926473e-17, 8.16553346980254e-17, 9.518438074851597e-17, 4.5732435423776064e-17, 5.064614036026238e-17, 7.54185622781279e-17, 5.4283787105467464e-17, 1.0535466930190695e-16, 6.015929298492577e-17, 7.540167836380345e-17, 5.549055174946024e-17, 4.761028094336883e-17, 6.641347920748904e-17, 5.699383489544804e-17, 4.5141984446337134e-17, 1.0051614597061523e-16, 9.101396977540696e-17, 5.715241670412556e-17, 9.211808755707206e-17, 7.315239856585862e-17, 6.959523293922953e-17, 4.6766695280116283e-17, 6.318250118118892e-17, 5.385406658709943e-18, 9.954070811828397e-17, 7.707258639571146e-17, 6.601781260307118e-17, 3.6616687221720154e-17, 6.31043901925118e-17, 6.52952967686394e-17, 1.462709138087262e-16, 9.488676156771198e-17, 8.232519391546

## 8: Traces
- (8a): find $D = C + \mathrm{trc}(C)\mathbf{1} = D_0\mathbf{1} + \vec{D} * \vec{\sigma}$
- (8b): Test the equivalent of $(D_0, \vec{D})$ and $D = C + \mathrm{trc}(C)\mathbf{1}$

In [21]:
##8a
c0xyz = rho2timesrho4(p0xyz_2, p0xyz_4) #arrays were generated in cell 18
print('check C array:', c0xyz)

check C array: [-0.5, 0j, 0j, 0j]


In [22]:
C = final_C_1(p0xyz_2, p0xyz_4) #can use either function c1 or c2 both make the matrix c
print ('check matrix C:', C)

check matrix C: [[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]


In [23]:
trC = np.trace(C)
print('trace of C=', trC, '2C_0=', 2*c0xyz[0])

identity_matrix = np.zeros((2,2), dtype = 'complex128')
identity_matrix[0,0]= 1
identity_matrix[1,1] = 1

trace of C= (-1+0j) 2C_0= -1.0


In [24]:
D = C + np.trace(C)*identity_matrix
print('check matrix D:', D)

check matrix D: [[-1.5+0.j  0. +0.j]
 [ 0. +0.j -1.5+0.j]]


In [25]:
#making matrix D another way: takes matrix D and produces d0xyz?
D2 = C + 2*c0xyz[0]
print (D)
print (D2)

[[-1.5+0.j  0. +0.j]
 [ 0. +0.j -1.5+0.j]]
[[-1.5+0.j -1. +0.j]
 [-1. +0.j -1.5+0.j]]


## 9: Creating the real functions!
- create functions to calculate E and F:
- $E = \rho^{(3)}\big[(\mathbf{1} -\rho^{(2)})\rho^{(4)} + \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)\big] $
    - $E = \rho^{(3)} \big[C + \mathrm{tr}(C)\big] $
- $F = (\mathbf{1} - \rho^{(1)}) \rho^{(3)}\big[(\mathbf{1} -\rho^{(2)})\rho^{(4)} + \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)\big] $
    - $F = (\mathbf{1} - \rho^{(1)}) \rho^{(3)}\big[C + \mathrm{tr}(C)\big] $

#### THE BIG IDEA:

so the idea will be to slowly build up to function F
- first we create $(\mathbf{1} -\rho^{(2)})\rho^{(4)}$, which we call C
    - call $\mathbf{rho2timesrho4}$(p0xyz_2, p0xyz_4) and it returns c0xyz
    - the you would do matrix C = $\textbf{matrix_maker}$(c0xyz)
    - OR call $\textbf{final_C_1}$(p0xyz_2, p0xyz_4) 
    - ***** use final c1 or final c2?

- then we will create $(\mathbf{1} -\rho^{(2)})\rho^{(4)}+ \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)$, which we will call D (just adding trace * identity onto C)
- then we will multiply matrix $\rho^{(3)}$, which we will call E
- lastly, we will multiply matrix $(\mathbf{1} -\rho^{(1)})$, which is then F

- Input: $P^{(1)}$, $P^{(2)}$, $P^{(3)}$, $P^{(4)}$, which I call p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4
- (10a): Output: $(2\mathrm{Re}[F_0], 2\mathrm{Re}[\vec{F}])$

In [26]:
# define final_C_1 if i change the function name...
# define identity_matrix
# need matrix_maker
# need to define P vectors
p0xyz_1 = [1, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]

In [27]:
## need new name for this
def matrix_mult_arrays(m10xyz, m20xyz): #arrays to array
    M10, M1x, M1y, M1z = m10xyz
    M20, M2x, M2y, M2z = m20xyz
    m1xyz = np.array([M1x, M1y, M1z])
    m2xyz = np.array([M2x, M2y, M2z])
    M30 = M10*M20 + np.tensordot(m1xyz, m2xyz, axes=1)
    m3xyz = M10*m2xyz + M20*m1xyz +1j*(np.cross(m1xyz, m2xyz))
    m30xyz = ([M30, m3xyz[0], m3xyz[1], m3xyz[2]])
    return m30xyz

In [28]:
c0xyz =[2, 1, 1, 1]
d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
print (d0xyz)

[6, 1, 1, 1]


In [29]:
## okay so we are going to make 2 functions to calculate F.... 
## one does the RHS which does matrix all the way
## the other does the LHS which keeps arrays all the way

#this function is for testing
def F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = final_C_1(p0xyz_2, p0xyz_4)
    #print ('C=', C)
    D = C + np.trace(C)*identity_matrix
    #print ('D=', D)
    E = matrix_maker(p0xyz_3)@D
    #print ('E=', E)
    F = matrix_maker(p0xyz_1)@E
    return (F)

#this function will go into code
def F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    #convert p's to a and b - what function does this? array_for_rho2 and array_for_rho4
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
   
    #multiply a and b
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)

    #add d
    d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
   
    #multiply p0xyz_3 to get e
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
   
    #multiply p0xyz_1 to get f
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
   
    #make array matrix
    F = matrix_maker(f0xyz)
    return (F)

In [30]:
p0xyz_1 = [1, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]

print ('INPUT P ARRAYS')
print('P1=', p0xyz_1)
print('P2=', p0xyz_2)
print('P3=', p0xyz_3)
print('P4=', p0xyz_4)

print ('CONVERTED ARRAYS')
print('a0xyz=', array_for_rho2(p0xyz_2))
print('b0xyz=', array_for_rho4(p0xyz_4))
print('c0xyz=', matrix_mult_arrays(a0xyz, b0xyz))

print ('MATRICES')
print ('A=',rho2_matrix(p0xyz_2))
print ('B=', rho4_matrix(p0xyz_4))

print ('F made with matrices', F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4))

print ('F made with arrays', F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4))

# compare both ways
FM = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
FA = F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print(type(FA))
print(type(FM))

INPUT P ARRAYS
P1= [1, 1, 1, 1]
P2= [1, 1, 1, 1]
P3= [1, 1, 1, 1]
P4= [1, 1, 1, 1]
CONVERTED ARRAYS
a0xyz= [0.5, -0.5, -0.5, -0.5]
b0xyz= [0.5, 0.5, 0.5, 0.5]
c0xyz= [-0.5, 0j, 0j, 0j]
MATRICES
A= [[ 0. +0.j  -0.5+0.5j]
 [-0.5-0.5j  1. +0.j ]]
B= [[1. +0.j  0.5-0.5j]
 [0.5+0.5j 0. +0.j ]]
F made with matrices [[-9.+0.j -3.+3.j]
 [-3.-3.j -3.+0.j]]
F made with arrays [[-9.+0.j -3.+3.j]
 [-3.-3.j -3.+0.j]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [31]:
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        random_numbers3 = random_complex_numbers()
        random_numbers4 = random_complex_numbers()

        #compute density matrix both ways
        L = F_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)
        M = F_array_to_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2, random_numbers3, random_numbers4)) #called it something different because i was trying to append random numbers to itslef and it said no
   
    return delta_values, random_numbers_used

In [32]:
N = 100
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [1.359732678904188e-16, 9.80956882511735e-17, 1.1411922612107033e-16, 7.510147882688836e-17, 1.1110937719382034e-16, 9.288328134094835e-17, 6.565349887594144e-17, 8.632185097439803e-17, 6.74329872132253e-17, 7.223585937339729e-17, 1.6444958025881714e-16, 7.388269176542139e-17, 1.1745921157912937e-16, 9.866131561538058e-17, 7.566826135658576e-17, 7.200772725832843e-17, 5.730053889078383e-17, 1.269662356102148e-16, 1.4768746113928098e-16, 1.6339133468566702e-16, 1.031846614408851e-16, 1.665660300767496e-16, 1.1425460794863419e-16, 1.9843245209257154e-16, 7.560817484377557e-17, 8.382923127412742e-17, 8.855392587858196e-17, 1.0425511975700307e-16, 1.1931038382688114e-16, 7.980115610728138e-17, 8.559155046266528e-17, 1.2565961689169644e-16, 1.9786888628078128e-16, 1.0452058131929304e-16, 4.0662803745354896e-17, 8.697733929428938e-17, 9.194915178988473e-17, 1.0251563886130425e-16, 6.635912820635889e-17, 1.0365481163563107e-16, 1.0272792369943448e-16, 8.531103953976046e-17, 1.66

## 10: Function from Bennett et al
- make function that inputs the 4 p arrays and outputs an array of REAL NUMBERS so output: $(2\mathrm{Re}[F_0], 2\mathrm{Re}[\vec{F}])$

testing complex conjugate stuff:

In [33]:
# found np.conjugate(): function that takes the complex conjugate
z = 3 + 2*1j
z_conj = np.conjugate(z)
print(z, z_conj)

(3+2j) (3-2j)


In [34]:
z = [1, -1j, 1j, 1]
z_conj = np.conjugate(z)
print(z, z_conj)

[1, (-0-1j), 1j, 1] [ 1.-0.j -0.+1.j  0.-1.j  1.-0.j]


In [35]:
def F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4): #same function as before but minus making the matrix at the end
    a0xyz = array_for_rho2(p0xyz_2) # (1-rho) 
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)
    d0xyz = np.array([c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]])
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
    return (f0xyz)

def F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = final_C_1(p0xyz_2, p0xyz_4)
    D = C + np.trace(C)*identity_matrix
    E = matrix_maker(p0xyz_3)@D
    F = matrix_maker(p0xyz_1)@E
    return (F)

p0xyz_1 = [1j, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, -1j]

f0xyz = F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
f0xyz_conj = np.conjugate(f0xyz)

print('ARRAY STUFF')
print('f0xyz=',f0xyz)
print('f0xyz_conj=',f0xyz_conj)
Re_f0xyz = f0xyz + f0xyz_conj
print('Re_f0xyz', Re_f0xyz)

Re_F_made_w_arrays = matrix_maker(Re_f0xyz)
print(Re_F_made_w_arrays)

F = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
F_conj = np.conjugate(F)

print('MATRIX STUFF')
print('F=',F)
print('F_conj=',F_conj)
Re_F = F + F_conj
print('Re_F=',Re_F)

ARRAY STUFF
f0xyz= [(-3+1j), (-2+0j), (-1-1j), (-2+0j)]
f0xyz_conj= [-3.-1.j -2.-0.j -1.+1.j -2.-0.j]
Re_f0xyz [-6.+0.j -4.+0.j -2.+0.j -4.+0.j]
[[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]
MATRIX STUFF
F= [[-5.+1.j -3.+1.j]
 [-1.-1.j -1.+1.j]]
F_conj= [[-5.-1.j -3.-1.j]
 [-1.+1.j -1.-1.j]]
Re_F= [[-10.+0.j  -6.+0.j]
 [ -2.+0.j  -2.+0.j]]


In [36]:
## 10a:
# the funciton as is outputs the F matrix, we want to take that and
# take the real part and print out the array of that
def Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0xyz_conj = np.conjugate(f0xyz)
    Re_f0xyz = f0xyz + f0xyz_conj
    return Re_f0xyz

def Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    return Fvvsc

def arr_2_mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    Re_f0xyz = Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    Fvvsc_from_array = matrix_maker(Re_f0xyz)
    return Fvvsc_from_array

In [37]:
p0xyz_1 = [1j, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, -1j]


Re_f0xyz = Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
Fvvsc_from_array = matrix_maker(Re_f0xyz)
print('Fvvsc_array PRODUCES:')
print(Fvvsc_from_array)

Fvvsc_arr_2_mat = arr_2_mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print('arr_2_mat PRODUCES:')
print(Fvvsc_arr_2_mat)

F_made_with_matrices = Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print('Fvvsc_matrix PRODUCES:')
print(F_made_with_matrices)

Fvvsc_array PRODUCES:
[[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]
arr_2_mat PRODUCES:
[[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]
Fvvsc_matrix PRODUCES:
[[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]


In [38]:
## testing
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        random_numbers3 = random_complex_numbers()
        random_numbers4 = random_complex_numbers()

        #compute density matrix both ways   
        L = arr_2_mat(random_numbers1, random_numbers2, random_numbers3, random_numbers4)
        M = Fvvsc_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2, random_numbers3, random_numbers4)) #called it something different because i was trying to append random numbers to itslef and it said no
   
    return delta_values, random_numbers_used

In [39]:
N = 2
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [1.4879533090907162e-16, 2.6899768912808212e-17]
RANDOM NUMBERS USED: [(array([ 5.38618665e-01-0.42338536j, -2.27824357e+00+1.10377203j,
       -5.08552329e-01-0.19915687j,  1.89148572e-03-0.71744811j]), array([-0.05097014-1.70471788j, -0.27785914+0.15605253j,
        0.88788379+1.55386937j, -0.37578682+0.54942258j]), array([ 2.36452878-2.00316459j, -0.48467361+0.90560567j,
       -0.7108738 -0.67752194j, -0.27243651-1.01990749j]), array([ 0.15197224+0.54574956j, -0.55115003+0.67075433j,
        1.46330285+0.59356364j,  0.70548494-0.22595746j])), (array([ 0.20074312-0.41908766j, -1.17035062-0.40964558j,
       -0.83069725-0.36191616j, -0.86637764+0.63298777j]), array([ 0.23879721-0.64034048j, -0.37379016+0.84171317j,
       -0.63638854+2.50216022j,  1.09860394+0.95000947j]), array([-1.06536215-0.40464962j, -0.15946639+0.4677045j ,
        0.53781035-0.43277171j,  1.28566525-0.16487515j]), array([-0.6028927 -0.07012823j,  0.56811444-1.21301518j,
        0.78029747+0.184372

In [40]:

#two ways to make final F
def F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4): #same function as before but minus making the matrix at the end
    a0xyz = p0xyz2_to_a0xyz(p0xyz_2)
    b0xyz = p0xyz4_to_b0xyz(p0xyz_4)
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)
    d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
    return (f0xyz)
#note: can probably refine F_array_to_matrix function to just call F_array and then matrix maker
def F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    a0xyz = p0xyz2_to_a0xyz(p0xyz_2)
    b0xyz = p0xyz4_to_b0xyz(p0xyz_4)
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)
    d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
    F = matrix_maker(f0xyz)
    return (F)

def F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = p0xyz2_p0xyz4_to_C(p0xyz_2, p0xyz_4)
    D = C + np.trace(C)*identity_matrix
    E = matrix_maker(p0xyz_3)@D
    F = matrix_maker(p0xyz_1)@E
    return (F)

# RENAMING THINGS-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [49]:
#matrix 1 array, matrix 2 array --> output matrix 3 array

######### KEY:
######### takes two, 4 component vectors each corresponding to a matrix
######### multiplies the "matrices" but keeps them as vector/array representations the whole time
######### outputs the components for the resulting matrix
def multiply_matrices(M1components, M2components): 
    M10, M1x, M1y, M1z = M1components
    M20, M2x, M2y, M2z = M2components
    m1xyz = np.array([M1x, M1y, M1z])
    m2xyz = np.array([M2x, M2y, M2z])
    M30 = M10*M20 + np.tensordot(m1xyz, m2xyz, axes=1)
    m3xyz = M10*m2xyz + M20*m1xyz +1j*(np.cross(m1xyz, m2xyz))
    M3components = np.array([M30, m3xyz[0], m3xyz[1], m3xyz[2]])
    return M3components

#functions to take p arrays and make general a and b, so that we can do multiplication noramllly

######### KEY:
######### this function generalizes the specific array that is (identity minus rho)...
######### it takes the 4 component vector/array that would create a matrix that is in the form (1-rho)
######### makes it into more generalized components "a0xyz"
def generalize_IDrho(IDrho_array):
    P0, Px, Py, Pz = IDrho_array
    pxyz = np.array([Px, Py, Pz])
    A0 = 1 - 0.5*P0
    axyz = -0.5*P0*pxyz
    a0xyz = np.array([A0, axyz[0], axyz[1], axyz[2]])
    return a0xyz #makes array to be put into matrix maker to create (ID-rho)

######### KEY:
######### this function generalizes the specific array that is just rho...
######### it takes the 4 component vector/array that would create a matrix that is in the form rho
######### makes it into more generalized components "b0xyz"
def generalize_rho(rho_array):
    P0, Px, Py, Pz = rho_array
    pxyz = np.array([Px, Py, Pz])
    B0 = 0.5*P0
    bxyz = 0.5*P0*pxyz
    b0xyz = np.array([B0, bxyz[0], bxyz[1], bxyz[2]])
    return b0xyz #makes array to be put into matrix maker to create rho

######### KEY:
######### this function generalizes then multiplies:
######### first input is rho matrix that is identity minus rho (1-rho)
######### second input is rho matrix that is solo (rho)
######### takes generalized versions and multiplies them in the order (1-rho)rho
def generalize_then_mult(IDrho_array, rho_array):
    a0xyz = generalize_IDrho(IDrho_array)
    b0xyz = generalize_rho(rho_array)
    c0xyz = multiply_matrices(a0xyz, b0xyz)
    C = matrix_maker(c0xyz)
    return C

######### KEY:
######### this function generalizes then multiplies:
######### first input is rho matrix that is identity minus rho (1-rho)
######### second input is rho matrix that is solo (rho)
######### NOTE THE DIFFERENCE:
######### this function takes generalized versions and multiplies them in the order rho(1-rho)
def generalize_then_mult_term2(IDrho_array, rho_array):
    a0xyz = generalize_IDrho(IDrho_array)
    b0xyz = generalize_rho(rho_array)
    c0xyz = multiply_matrices(b0xyz, a0xyz)
    C = matrix_maker(c0xyz)
    return C

In [50]:
#two ways to make final F (term 1) 
#NOTE: these are all not the final Fvvsc, because they have not yet added the harmonic conjugate

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the first term in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)]
######### it outputs the resulting matrix for this term, but as its ARRAY/VECTOR representation
def F_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    # (1-rho2)
    a0xyz = generalize_IDrho(p0xyz_2)
    
    # rho4
    b0xyz = generalize_rho(p0xyz_4)
    
    # (1-rho2)rho4 ....aka....
    # a0xyz * b0xyz
    c0xyz = multiply_matrices(a0xyz, b0xyz)
    
    # (1-rho2)rho4 + tr(-)
    d0xyz = np.array([c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]])
    
#now need to generalize rho3 and rho1 arrays
    
    # (1-rho1)
    g0xyz = generalize_IDrho(p0xyz_1)
    
    # rho3
    h0xyz = generalize_rho(p0xyz_3)
    
    # e0xyz = rho3[(1-rho2)rho4 + tr(-)] .... aka.....
    # e0xyz = h0xyz * d0xyz
    e0xyz = multiply_matrices(h0xyz, d0xyz)
    
    # f0xyz = (1-rho1)rho3[(1-rho2)rho4 + tr(-)] .... aka.....
    # f0xyz = g0xyz * h0xyz * d0xyz .... aka.....
    # f0xyz = g0xyz * e0xyz
    f0xyz = multiply_matrices(g0xyz, e0xyz)
    return (f0xyz)

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the first term in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)]
######### it does this by using the components created from "F_components_term1" function then putting it into matrix_maker
def F_array_to_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F = matrix_maker(f0xyz)
    return (F)

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the first term in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)]
######### it outputs the resulting MATRIX for this term
def F_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    # C = (1-rho2)rho4 ..... or.....
    # C = A*B 
    C = generalize_then_mult(p0xyz_2, p0xyz_4)
    
    # D = (1-rho2)rho4 + tr((1-rho2)rho4)
    # D = C + tr(C)*I
    D = C + np.trace(C)*identity_matrix
    
    # need to generalize rho3 and rho1 arrays
    # (1-rho1) 
    g0xyz = generalize_IDrho(p0xyz_1)
    
    # rho1
    h0xyz = generalize_rho(p0xyz_3)
    
    # E = rho3[(1-rho2)rho4 + tr((1-rho2)rho4)]
    # E = rho3*[C + tr(C)*I]
    # E = rho3*D
    E = matrix_maker(h0xyz)@D
    
    # F = (1-rho1)rho3[(1-rho2)rho4 + tr((1-rho2)rho4)]
    # F = (1-rho1)rho3*[C + tr(C)*I]
    # F = (1-rho1)rho3*D
    # F = (1-rho1)*E
    F = matrix_maker(g0xyz)@E
    return (F)

## 11: Repeat but for the other term
- (11a): create another function: switch 2 and 4 and switch 1 and 3
$F = (\mathbf{1} - \rho^{(1)}) \rho^{(3)}\big[(\mathbf{1} -\rho^{(2)})\rho^{(4)} + \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)\big]  - \rho^{(1)}(\mathbf{1} - \rho^{(3)})\big[\rho^{(2)}(\mathbf{1} -\rho^{(4)}) + \mathrm{tr}\big(\rho^{(2)}(\mathbf{1} -\rho^{(4)})\big)\big] + \mathrm{h.c.} $
- (11b): test it
(we will do this for all the terms in A.11)

In [51]:
######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the second term in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)]
######### it outputs the resulting matrix for this term, but as its ARRAY/VECTOR representation
def F_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4): #same function as before but minus making the matrix at the end
    # (1-rho4)
    a0xyz = generalize_IDrho(p0xyz_4)
    
    ######should i do it like a is always the lower rho number, or should 
    ######i do it like a is always the first matrix
    ######or like a is always the identity one (this is what I have now)--- DECIDED TO KEEP THIS WAY
    
    # rho2
    b0xyz = generalize_rho(p0xyz_2)
    
    # rho2(1-rho4)
    c0xyz = multiply_matrices(b0xyz, a0xyz)
    
    # rho2(1-rho4) + tr(-)
    d0xyz = np.array([c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]])

    ###generalize rho3 and rho1 
    
    # (1-rho3) 
    g0xyz = generalize_IDrho(p0xyz_3)
    
    # rho1
    h0xyz = generalize_rho(p0xyz_1)

    # e0xyz = (1-rho3)[rho2(1-rho4) + tr(-)]
    # e0xyz = g0xyz * d0xyz
    e0xyz = multiply_matrices(g0xyz, d0xyz)
    
    # f0xyz = rho1(1-rho3)[rho2(1-rho4) + tr(-)]
    # f0xyz = h0xyz * g0xyz * d0xyz
    # f0xyz = h0xyz * e0xyz
    f0xyz = multiply_matrices(h0xyz, e0xyz)
    return (f0xyz)

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the second term in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)]
######### it does this by using the components created from "F_components_term2" function then putting it into matrix_maker
def F_array_to_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F = matrix_maker(f0xyz)
    return (F)

                     
######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the second term in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)]
######### it outputs the resulting MATRIX for this term
def F_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    # C = rho2(1-rho4) ..... or.....
    # C = B*A 
    C = generalize_then_mult_term2(p0xyz_4, p0xyz_2) 
    
    # D = rho2(1-rho4) + tr(rho2(1-rho4))
    # D = C + tr(C)*I
    D = C + np.trace(C)*identity_matrix
    
    # need to generalize rho3 and rho1 arrays
    # (1-rho3) 
    g0xyz = generalize_IDrho(p0xyz_3)
    
    # rho1
    h0xyz = generalize_rho(p0xyz_1)
    
    # E = (1-rho3)[rho2(1-rho4) + tr(rho2(1-rho4))]
    # E = (1-rho3)*[C + tr(C)*I]
    # E = (1-rho3)*D
    E = matrix_maker(g0xyz)@D
    
    # F = rho1(1-rho3)[rho2(1-rho4) + tr(rho2(1-rho4))]
    # F = rho1(1-rho3)*[C + tr(C)*I]
    # F = rho1(1-rho3)*D
    # F = rho1*E
    F = matrix_maker(h0xyz)@E
    return (F)

## Fvvsc with new function names

In [52]:
#NEW FUNCTION NAMES FVVSC TERM 1
# now adding in the harmonic conjugates

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the FIRST TERM in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)] using F_components_term1
######### then it adds the conjugate of this array, adds it to the original array, and outputs the real array
def Fvvsc_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0xyz_conj = np.conjugate(f0xyz)
    Re_f0xyz = f0xyz + f0xyz_conj
    return Re_f0xyz

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the FIRST TERM in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)] using F_matrix_term1
######### then it adds the harmonic conjugate (conjugate and transpose) of this matrix, adds it to the original matrix, and outputs the hermitian matrix
def Fvvsc_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    return Fvvsc

######## KEY:
######## takes output array from Fvvsc_components_term1 and makes matrix
def Fvvsc_arr2mat_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    Re_f0xyz = Fvvsc_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    Fvvsc_from_array = matrix_maker(Re_f0xyz)
    return Fvvsc_from_array

In [53]:
#NEW FUNCTION NAMES FVVSC TERM 2

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the SECOND TERM in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)] using F_components_term2
######### then it adds the harmonic conjugate of this array, adds it to the original array, and outputs the real array
def Fvvsc_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0xyz_conj = np.conjugate(f0xyz)
    Re_f0xyz = f0xyz + f0xyz_conj
    return Re_f0xyz

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the SECOND TERM in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)] using F_matrix_term2
######### then it adds the harmonic conjugate (conjugate and transpose) of this matrix, adds it to the original matrix, and outputs the hermitian matrix
def Fvvsc_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    return Fvvsc

######## KEY:
######## takes output array from Fvvsc_components_term2 and makes matrix
def Fvvsc_arr2mat_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    Re_f0xyz = Fvvsc_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    Fvvsc_from_array = matrix_maker(Re_f0xyz)
    return Fvvsc_from_array

## FULL Fvvsc, BOTH TERMS

In [54]:
#now for the full term!

######## KEY:
######## this creates the full Bennet A.11 equation in ARRAY/VECTOR/COMPONENT form:
######## it does this by taking in those 4 arrays corresponding to 4 matrices
######## and then it uses Fvvsc_components_term1 and Fvvsc_components_term2 
######## to output the full array/vector representatation of the full Fvvsc term
def Fvvsc_components(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    term1 = Fvvsc_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    term2 = Fvvsc_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    return term1 + term2

######## KEY:
######## this creates the full Bennet A.11 equation in MATRIX form: 
######## it does this by taking in those 4 arrays corresponding to 4 matrices
######## and then it uses Fvvsc_matrix_term1 and Fvvsc_matrix_term2 
######## to output the full MATRIX of the full Fvvsc term
def Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    term1 = Fvvsc_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    term2 = Fvvsc_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    return term1 + term2

######## KEY:
######## uses Fvvsc_arr2mat_term1 and Fvvsc_arr2mat_term2 to make the full MATRIX of the full Fvvsc term
def Fvvsc_arr2mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    term1 = Fvvsc_arr2mat_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    term2 = Fvvsc_arr2mat_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    return term1 + term2

In [55]:
p0xyz_1 = [1j, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, -1j]

def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        random_numbers3 = random_complex_numbers()
        random_numbers4 = random_complex_numbers()

        #compute density matrix both ways   
        L = Fvvsc_arr2mat(random_numbers1, random_numbers2, random_numbers3, random_numbers4)
        M = Fvvsc_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2, random_numbers3, random_numbers4)) #called it something different because i was trying to append random numbers to itslef and it said no
   
    return delta_values, random_numbers_used

In [58]:
N = 100
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [6.828198373677918e-17, 1.0005635024722055e-16, 6.9857955436089e-17, 5.827523635720515e-17, 1.300494833090566e-16, 4.3868813174864226e-16, 2.509675033770437e-16, 1.3842740209173595e-16, 7.042965969575413e-17, 3.968734452437547e-17, 1.9574605026615555e-16, 7.075756863897512e-17, 9.405138894073724e-17, 7.465496420409764e-17, 1.0969293825367114e-16, 1.4915378894827376e-16, 1.51815609460929e-16, 7.78242863563455e-16, 6.950173634511632e-17, 2.2217355088660543e-16, 1.1494904663176832e-16, 2.408602632695887e-16, 2.489997178368193e-16, 7.460544289514658e-17, 4.8188132551885795e-17, 8.86446542346959e-17, 7.903980334536495e-17, 4.827822986205166e-17, 1.2834009530649693e-16, 7.912420951240559e-17, 4.200331331529573e-17, 2.4647091531438048e-17, 9.570134838005215e-17, 1.9520927085513112e-16, 6.065720848002003e-17, 1.0553312960572365e-16, 6.785374109906706e-17, 6.260238959278434e-17, 5.708393214434135e-17, 9.756659849700284e-17, 9.101281405745696e-17, 1.9809710438507157e-16, 1.06497899